In [1]:
from importlib.metadata import version

import tiktoken
import torch
import re

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))
import os
os.environ['TIKTOKEN_CACHE_DIR'] = ""


torch version: 2.3.0
tiktoken version: 0.7.0


In [2]:
with open('the-verdict.txt', 'r', encoding="utf-8") as f:
    raw_text = f.read()
print("Total number of characters in file: ", len(raw_text))
print(raw_text[:99])

Total number of characters in file:  20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


Tokenizing

In [3]:
# convert raw text file into a list of strings (tokens)
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])
print(len(preprocessed))

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']
4649


Converting to Token ID

In [4]:
unique_words = sorted(list(set(preprocessed)))
print(len(unique_words))

1159


In [5]:
# create a vocabulary using all unique words
# sorted in alphabetic order
vocab = {token:integer for integer,token in enumerate(unique_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Carlo;', 25)
('Chicago', 26)
('Claude', 27)
('Come', 28)
('Croft', 29)
('Destroyed', 30)
('Devonshire', 31)
('Don', 32)
('Dubarry', 33)
('Emperors', 34)
('Florence', 35)
('For', 36)
('Gallery', 37)
('Gideon', 38)
('Gisburn', 39)
('Gisburns', 40)
('Grafton', 41)
('Greek', 42)
('Grindle', 43)
('Grindle:', 44)
('Grindles', 45)
('HAD', 46)
('Had', 47)
('Hang', 48)
('Has', 49)
('He', 50)


In [6]:
#combining encoding and decoding
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text


In [7]:
# adding unkown tokens
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}
print(len(all_tokens))
#print last 5 items in vocabulary
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

1161
('younger', 1156)
('your', 1157)
('yourself', 1158)
('<|endoftext|>', 1159)
('<|unk|>', 1160)


In [8]:
# updating tokenizer to read unknown vocab
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int 
                        else "<|unk|>" for item in preprocessed]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [9]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [10]:
tokenizer.encode(text)

[1160,
 5,
 362,
 1155,
 642,
 1000,
 10,
 1159,
 57,
 1013,
 981,
 1009,
 738,
 1013,
 1160,
 7]

In [11]:
tokenizer.decode(tokenizer.encode(text))
# hello not in vocab - decoded as unknown

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

Byte Pair Encoding

In [12]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))


tiktoken version: 0.7.0


In [13]:
import os
import tempfile
from pathlib import Path
import shutil

# Set the environment variable to prevent caching
os.environ['TIKTOKEN_CACHE_DIR'] = ""

# Identify the cache directory
cache_dir = os.path.join(tempfile.gettempdir(), "data-gym-cache")
print(f"Cache directory: {cache_dir}")

# Check if the cache directory exists and clear it if it does
if os.path.exists(cache_dir):
    print("Cache directory found. Attempting to clear it...")
    shutil.rmtree(cache_dir)
    print("Cache directory cleared.")
else:
    print("Cache directory does not exist.")


Cache directory: /tmp/data-gym-cache
Cache directory does not exist.


In [14]:


import tiktoken


tokenizer = tiktoken.get_encoding("gpt2")


In [15]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271, 13]


In [16]:
strings = tokenizer.decode(integers)
print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace.


In [17]:
text = ("Akwirw ier")
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)
strings = tokenizer.decode(integers)
print(strings)

[33901, 86, 343, 86, 220, 959]
Akwirw ier


Data Sampling - Sliding Window

In [18]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

encoded_text = tokenizer.encode(raw_text)
print(len(encoded_text))


5145


In [19]:
# removing 50 tokens from dataset to create a sample dataset
encoding_sample = encoded_text[50:]

In [20]:
# next word prediction
# x contains inputs
# y contains targets shifter by one position

context_size = 4
x = encoding_sample[:context_size]
y = encoding_sample[1:context_size+1]

print(x,y)

[290, 4920, 2241, 287] [4920, 2241, 287, 257]


In [21]:
# next word prediction:
for i in range(1, context_size+1):
    context = encoding_sample[:i]
    desired = encoding_sample[i]
    print(context, "--->", desired)

[290] ---> 4920
[290, 4920] ---> 2241
[290, 4920, 2241] ---> 287
[290, 4920, 2241, 287] ---> 257


Data Loader

In [22]:
# extracts chunks from the input dataset, and creates another dataset and dataloader
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.tokenizer = tokenizer
        self.input_ids = [] #input data ID
        self.target_ids = [] # target ID (to be predicted)

        # tokenize
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # sliding window to extract chunks of max length into overlapping sequences
        for i in range(0, len(token_ids)-max_length, stride): # subtracting max length to prevent out of bound error
            input_chunk = token_ids[i: i+max_length]
            target_chunk = token_ids[i+1: i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, index):
        return self.input_ids[index], self.target_ids[index]
        

In [23]:
def create_dataloader_1(txt, batch_size = 4, max_length = 256, stride=128, shuffle = True, num_workers = True, drop_last = True):
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=0
    )

    return dataloader

In [24]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [35]:
dataloader = create_dataloader_1(
    raw_text, batch_size=8, max_length=4, stride=4, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]]), tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])]


In [36]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[  287,   262,  6001,   286],
        [  465, 13476,    11,   339],
        [  550,  5710,   465, 12036],
        [   11,  6405,   257,  5527],
        [27075,    11,   290,  4920],
        [ 2241,   287,   257,  4489],
        [   64,   319,   262, 34686],
        [41976,    13,   357, 10915]]), tensor([[  262,  6001,   286,   465],
        [13476,    11,   339,   550],
        [ 5710,   465, 12036,    11],
        [ 6405,   257,  5527, 27075],
        [   11,   290,  4920,  2241],
        [  287,   257,  4489,    64],
        [  319,   262, 34686, 41976],
        [   13,   357, 10915,   314]])]


Token Embeddings

In [39]:
input_ids = torch.tensor([2,3,5,1])
vocab_size = 6 # number of unique tokens in the vocabulary
output_size = 3 # token embedding dimension

An embedding layer is a lookup table corresponding to embedded vectors of token IDs

In [43]:
torch.manual_seed(12) # for reproducibility
embedding = torch.nn.Embedding(vocab_size, output_size)
print(embedding.weight) # prints the embedding weights - random values

Parameter containing:
tensor([[-0.1320, -0.1254, -0.1610],
        [-0.8372, -1.2945, -0.2623],
        [-0.5353,  1.3466, -0.6958],
        [-0.0522,  0.4924, -0.6921],
        [-0.6944, -0.2125,  0.2362],
        [-1.2438,  1.4400,  0.7946]], requires_grad=True)


In [45]:
print(embedding(input_ids)) # prints the embedding of the input_ids
# since ids are 2,3,5,1, the embedding will be the 2nd, 3rd, 5th and 1st row of the embedding matrix

tensor([[-0.5353,  1.3466, -0.6958],
        [-0.0522,  0.4924, -0.6921],
        [-1.2438,  1.4400,  0.7946],
        [-0.8372, -1.2945, -0.2623]], grad_fn=<EmbeddingBackward0>)


In [46]:
print(embedding(torch.tensor([2])))

tensor([[-0.5353,  1.3466, -0.6958]], grad_fn=<EmbeddingBackward0>)


Position Aware Embeddings

In [51]:
# bigger vocab size
vocab_size = 50257 #rows
output_size = 256 #hyperparameter
token_embedding_layer = torch.nn.Embedding(vocab_size, output_size)
token_embedding_layer.weight.shape

torch.Size([50257, 256])

In [54]:
max_length = 4
dataloader = create_dataloader_1(raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)
# prints 8 text samples with 4 tokens each

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [56]:
token_embeddings = token_embedding_layer(inputs)
token_embeddings.shape
# 8 batches or sentences, 4 tokens (4 words in each sentence), 256 embedding dimensions

torch.Size([8, 4, 256])

In [59]:
context_length = max_length
position_embedding_layer = torch.nn.Embedding(context_length, output_size) 
print(position_embedding_layer.weight)
position_embeddings = position_embedding_layer(torch.arange(context_length)) # 0,1,2,3
print(position_embeddings)
print("Position embeddings shape:\n", position_embeddings.shape)


Parameter containing:
tensor([[ 0.3195,  0.3639, -0.7911,  ...,  0.1796, -0.9414,  1.1070],
        [ 1.0398,  2.5067,  0.1492,  ...,  0.4365, -0.9908, -1.4451],
        [-0.6483, -0.0723,  0.2375,  ..., -0.6013, -0.1751, -0.0245],
        [-0.3891, -0.2731, -0.2301,  ..., -0.2332,  0.4701, -0.4688]],
       requires_grad=True)
tensor([[ 0.3195,  0.3639, -0.7911,  ...,  0.1796, -0.9414,  1.1070],
        [ 1.0398,  2.5067,  0.1492,  ...,  0.4365, -0.9908, -1.4451],
        [-0.6483, -0.0723,  0.2375,  ..., -0.6013, -0.1751, -0.0245],
        [-0.3891, -0.2731, -0.2301,  ..., -0.2332,  0.4701, -0.4688]],
       grad_fn=<EmbeddingBackward0>)
Position embeddings shape:
 torch.Size([4, 256])


Inputs to the LLM

In [61]:
input_embeddings = token_embeddings + position_embeddings
print("Input embeddings shape:\n", input_embeddings.shape)

Input embeddings shape:
 torch.Size([8, 4, 256])
